<a href="https://colab.research.google.com/github/nEXOSkyline/XPMSimulator/blob/main/analysis/newXPMdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first step will install packages needed to use ROOT and modify the PATH in order to tell python where to look for pyROOT. It will also create an example histogram to test that everything is working as expected. 

In [ ]:
!uname -a

In [ ]:
!wget https://root.cern/download/root_v6.24.08.Linux-ubuntu20-x86_64-gcc9.4.tar.gz
!tar -zxvf /content/root_v6.24.08.Linux-ubuntu20-x86_64-gcc9.4.tar.gz
!apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion libpython3.6-dev
import sys
sys.path.append("/content/root_build/")
sys.path.append("/content/root_build/bin/")
sys.path.append("/content/root_build/include/")
sys.path.append("/content/root_build/lib/")
import ctypes
ctypes.cdll.LoadLibrary('/content/root_build/lib//libCore.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libThread.so')
ctypes.cdll.LoadLibrary('/content/root_build/lib//libTreePlayer.so')
import ROOT




We now mount Google Drive locally (more info at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA)

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
! git clone https://github.com/nEXOSkyline/XPMSimulator

In [ ]:
! cd XPMSimulator && git remote update && git merge origin/main

In [ ]:
import sys, os
import glob
import numpy as np
import matplotlib as mpl
mpl.rc('figure',facecolor='white')
from matplotlib import pyplot as plt
from matplotlib import dates as dt
from collections import defaultdict
import seaborn as sns
from array import array


#import datetime
import ROOT
from ROOT import TCanvas, TPad, TFormula, TF1, TPaveLabel, TH1F, TFile, TPaveText
from ROOT import gROOT, gBenchmark
ROOT.gErrorIgnoreLevel = 3000
# Check version of python, works with two or three
#version = sys.version_info.major
def vtoa( buf, entries ):
  retarr = []
  for idx in range(0,entries-1) :
    retarr.append( buf[idx] )
  return retarr
#hardcoded stuff
#path = 'C:/Users/exouser/Desktop/xpm_fitter_data/'    
#path = './'    
#path = '/mnt/c/Users/exouser/Desktop/xpm_fitter_data/'    
#path = '/home/kolo/Downloads/kolos_stuff_from_lab_PC/'    
path = '/content/XPMSimulator/analysis/'
#path = 'https://github.com/shamondho/nEXOskyline/tree/main/XPM_analysis/data/'

#myhist = ROOT.TH2F()
#myhist.SetName('myhist')

tree = ROOT.TTree('xpmdata','')
t2 = ROOT.TTree('t2','')

es = 0
sim = array('f',[0.0])
my_list_of_files = glob.glob(path + 'standard*.txt')
tree.ReadFile(my_list_of_files[0],'Tc:Ta:TcRise:TaRise:cat:an:offst:datime:IR:UV:chi2:n1:n2:catll:catul:anll:anul',',')
print(tree.GetEntries())
#bb = t2.Branch("tau0", sim, "tau0/F")
#for j in range(es,es+tree.GetEntries()) :
#t2.Scan('tau0')

#set up the ROOT tree with the dataset from the file "control5int.txt"
#tree.ReadFile(path + 'control5int.txt','Tc:Ta:TcRise:TaRise:cat:an:offst:datime:IR:UV',',')

#You can add in the data from the next file by repeating 
#the ReadFile command with a new filename, eg

#file name example : /content/nEXOskyline/XPM_analysis/data/synth10ms14bit.txt
print('The total number of files is: ' + str(len(my_list_of_files)))
for index in range(1,len(my_list_of_files)):
  file_string = my_list_of_files[index]
  file_name = file_string[39:]
  print(file_string)
  tree.ReadFile(file_string,'Tc:Ta:TcRise:TaRise:cat:an:offst:datime:IR:UV:chi2:n1:n2:catll:catul:anll:anul',',')
  for j in range(es,es+tree.GetEntries()) :
    if file_string.find('1s')>=0 :
      sim[0] = (1.0e6)
    else :
      sim[0] = (float(my_list_of_files[4].split('sim')[1].split('.')[0]))
    t2.Fill()
  es = tree.GetEntries()
tree.AddFriend(t2)
#tree.Scan('cat:t2.tau0')
  
xsim = np.arange(0.0,5000.0,25.0)
ysim = np.arange(0.0,5000.0,25.0)
myhist = ROOT.TH2F('myhist','',1500,0.0,60000.0,1500,0.0,60000.0)
myhist2 = ROOT.TH2F('myhist2','',1500,0.0,60000.0,1500,0.0,60000.0)
myhist3 = ROOT.TH2F('myhist3','',1500,0.0,60000.0,1500,0.0,60000.0)

tree.Draw('(81.9-1.0)/log(cat/an):tau0>>myhist','tau0 < 5000','goff')
tree.Draw('(81.9-1.0)/log((cat-0.0045*cat)/(an+0.0041*an)):tau0>>myhist2','tau0 < 5000','goff')
tree.Draw('(81.9-1.0)/log((cat+0.0045*cat)/(an-0.0041*an)):tau0>>myhist3','tau0 < 5000','goff')

print(tree.GetEntries())
tau = vtoa(tree.GetV1(),tree.GetSelectedRows())
tau0 = vtoa(tree.GetV2(),tree.GetSelectedRows())

x = []
y = []
ely = []
ehy = []
myprof = myhist.ProfileX()
myprof2 = myhist2.ProfileX()
myprof3 = myhist3.ProfileX()


for bin in range(1,myprof.GetNbinsX()+2) :
  #print(bin,myprof.GetBinContent(bin),myprof.GetBinEntries(bin))
  if( myprof.GetBinEntries(bin)<=3 ): continue
  x.append(myprof.GetBinCenter(bin))
  y.append(myprof.GetBinContent(bin))
  ehy.append(myprof2.GetBinContent(bin) - myprof.GetBinContent(bin))
  ely.append(myprof.GetBinContent(bin) - myprof3.GetBinContent(bin))
errormatrix = np.array([np.array(ely),np.array(ehy)])
plt.errorbar(x,y,yerr=errormatrix,fmt='ro')
plt.ylabel('fitted $\\tau$ [$\mu$s]')
plt.xlabel('simulated $\\tau$ [$\mu$s]')
plt.plot(xsim,ysim,'-')
#f1=ROOT.TF1("f1","[0]+[1]*x",0,4)

#h2 = ROOT.h2
#h2.Fit(f1)
#h2.Draw()
#f1.Draw("same")
#canvas.Update()
#canvas.Print("test.pdf")

from ROOT import gROOT 
#gROOT.GetListOfCanvases().Draw()
            

In [ ]:
!sudo apt-get install libpython3.6-dev

In [ ]:
#using dictionaries to find the number of bins 
UV_dict = defaultdict(list) #used defaultdict to prevent dict from being classified as 'Nonetype' 
#logUV.sort()
 
for num in range(0,len(logUV)):
  x_val = logUV[num]

  UV_dict.setdefault(x_val, []).append(logcat[num]) #appending y values to specific key/x-value

new_x = list(UV_dict.keys())
x_bins = len(new_x)




In [ ]:
#all the files included in the section above 

#myhist = ROOT.TH2F('myhist','',50,1.5,3.0,50,1.3,2.0)

myhist = ROOT.TH2F('myhist','',x_bins,1.5,3.0,50,1.3,2.0) #using dictionary to find number of x bins 
#3rd parameter is number of bins in x direction, 4th is x-min and 5th is x-max
#same for y direction at 6th parameter
#print(tree.GetEntries())
tree.Draw('log10(cat):log10(UV)>>myhist','log10(UV)>1.0 && log10(cat)>1.25','goff')
       
 
myprof = myhist.ProfileX()

  
      
x = []
y = []
e = []
ex = []
for bin in range(1,myprof.GetNbinsX()+2) :
   # print(bin,myprof.GetBinContent(bin),myprof.GetBinEntries(bin))
   #if( myprof.GetBinEntries(bin)<=3 ): continue
   x.append(myprof.GetBinCenter(bin))
   y.append(myprof.GetBinContent(bin))
   e.append(myprof.GetBinError(bin))
   ex.append(myprof.GetBinWidth(bin)/2.0)
       
plt.errorbar(x,y,e)
plt.plot(logUV,logcat,'bo',markersize=1.5)
plt.ylabel('logcat')
plt.xlabel('logUV')
      


In [ ]:

plt.errorbar(x,y,e)

a, b = np.polyfit(x, y, 1)

#find method (stats-related) that provides confidence interval, or see if polyfit does that


for nums in range(0, len(x)):
    plt.plot(x[nums], a*x[nums]+b, 'mo', markersize=1.5)
    

print('Line of best fit: y = ' + str(a) + 'x + ' + str(b))
plt.ylabel('logcat')
plt.xlabel('logUV')


In [ ]:
#confidence interval

from scipy import stats
plt.errorbar(x,y,e)

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
#result = stats.linregress(x,y)

#confidence_interval = 2.58*std_err
confidence_interval = 1.703*std_err #1.703 is the t value I obtained from t-table for dof = 26, 95% confidence interval
for nums in range(0, len(x)):
   
   plt.plot(x[nums], a*x[nums]+b, 'mo', markersize=1.5)
   
   plt.plot(x[nums], (a+confidence_interval)*x[nums]+b, 'ro', markersize = 1.5) #upper limit 
   plt.plot(x[nums], (a-confidence_interval)*x[nums]+b, 'go', markersize = 1.5) #lower limit

plt.ylabel('logcat')
plt.xlabel('logUV')

print('The slope of the line of best fit is equal to ' + str(a) + ' '+ u"\u00B1" + ' '+ str(confidence_interval))

#need to update scipy to find confidence interval for intercept, should be an instance of object resulting from linregress() 

    

In [ ]:
#potential approach using dictionaries 
UV_dict = defaultdict(list) #used defaultdict to prevent dict from being classified as 'Nonetype' 
#logUV.sort()

for num in range(0,len(logUV)):
  x_val = logUV[num]

  UV_dict.setdefault(x_val, []).append(logcat[num]) #appending y values to specific key/x-value

#print(UV_dict.values())
new_x = list(UV_dict.keys()) #new x values/logUV values
new_y = list(UV_dict.values()) #new y values/logcat values

for i in range(0, len(new_y)):
    s = sum(new_y[i])
    l = len(new_y[i])
    average = s/l
    new_y[i] = average


plt.plot(new_x,new_y,'bo',markersize=1.5)

